In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import boto3
import string
import re

In [5]:
# Spam messages and count of total per category


data = [((1, u'Clothing_Shoes_and_Jewelry'), 6701), ((1, u'Toys_and_Games'), 3660), ((0, u'Electronics'), 243421), ((0, u'Sports_and_Outdoors'), 245343), ((1, u'Electronics'), 3175), ((0, u'Cell_Phones_and_Accessories'), 254268), ((1, u'Cell_Phones_and_Accessories'), 6283), ((0, u'Home_and_Kitchen'), 261986), ((1, u'Home_and_Kitchen'), 2255), ((1, u'Sports_and_Outdoors'), 8939), ((0, u'Toys_and_Games'), 250975), ((0, u'Clothing_Shoes_and_Jewelry'), 237228)]

In [6]:
working = []
for item in data:
    if item[0][0] == 1:
        working.append(['spam', item[0][1], item[1]])
    if item[0][0] == 0:
        working.append(['not spam', item[0][1], item[1]])
df_spam_cat_count_reviews = pd.DataFrame(working,columns = ["Spam","Category","Count_Reviews"])
df_spam_cat_count_reviews

,Spam,Category,Count_Reviews
0,spam,Clothing_Shoes_and_Jewelry,6701
1,spam,Toys_and_Games,3660
2,not spam,Electronics,243421
3,not spam,Sports_and_Outdoors,245343
4,spam,Electronics,3175
5,not spam,Cell_Phones_and_Accessories,254268
6,spam,Cell_Phones_and_Accessories,6283
7,not spam,Home_and_Kitchen,261986
8,spam,Home_and_Kitchen,2255
9,spam,Sports_and_Outdoors,8939


In [7]:
spam_overall = [((0, u'Home_and_Kitchen'), 2.8062301038986814), ((0, u'Clothing_Shoes_and_Jewelry'), 3.0823258637260356), ((0, u'Cell_Phones_and_Accessories'), 2.7989050922648544), ((1, u'Cell_Phones_and_Accessories'), 3.0741683908960686), ((0, u'Toys_and_Games'), 2.929220041836836), ((1, u'Home_and_Kitchen'), 3.4190687361419068), ((0, u'Sports_and_Outdoors'), 3.015366242362733), ((0, u'Electronics'), 2.7882310893472626), ((1, u'Sports_and_Outdoors'), 3.011522541671328), ((1, u'Toys_and_Games'), 3.5863387978142076), ((1, u'Electronics'), 3.0935433070866143), ((1, u'Clothing_Shoes_and_Jewelry'), 3.231308759886584)]

In [8]:
working = []
for item in spam_overall:
    if item[0][0] == 1:
        working.append(['spam', item[0][1], item[1]])
    if item[0][0] == 0:
        working.append(['not spam', item[0][1], item[1]])
df_spam_cat_overall_rating = pd.DataFrame(working,columns = ["Spam","Category","Overall_avg"])
df_spam_cat_overall_rating

,Spam,Category,Overall_avg
0,not spam,Home_and_Kitchen,2.806230
1,not spam,Clothing_Shoes_and_Jewelry,3.082326
2,not spam,Cell_Phones_and_Accessories,2.798905
3,spam,Cell_Phones_and_Accessories,3.074168
4,not spam,Toys_and_Games,2.929220
5,spam,Home_and_Kitchen,3.419069
6,not spam,Sports_and_Outdoors,3.015366
7,not spam,Electronics,2.788231
8,spam,Sports_and_Outdoors,3.011523
9,spam,Toys_and_Games,3.586339


In [16]:
df_comb = pd.merge(df_spam_cat_count_reviews, df_spam_cat_overall_rating, how='left', left_on =['Spam','Category'], right_on =['Spam','Category'])

In [77]:
df_comb_spam = df_comb[df_comb['Spam'] == 'spam']
df_comb_spam.Overall_avg

0    3.231309
1    3.586339
4    3.093543
6    3.074168
8    3.419069
9    3.011523
Name: Overall_avg, dtype: float64

In [76]:
df_comb_nonspam = df_comb[df_comb['Spam'] == 'not spam']
df_comb_nonspam.Overall_avg

2     2.788231
3     3.015366
5     2.798905
7     2.806230
10    2.929220
11    3.082326
Name: Overall_avg, dtype: float64

In [97]:
fig = go.Figure(layout=go.Layout(title=go.layout.Title(text='Approximately 3-4% of reviews are spam, with similar overall product ratings.')))
fig.add_trace(go.Bar\
             (name='not spam',x=df_comb_nonspam.Category, y=df_comb_nonspam.Count_Reviews,
              marker_color='skyblue'))
fig.add_trace(go.Bar\
             (name='spam',x=df_comb_spam.Category, y=df_comb_spam.Count_Reviews,
              marker_color='crimson'))

updatemenu = []
buttons = []

# button with one option for each dataframe
buttons.append(dict(method='restyle',
                        label='Number of Reviews',
                        visible=True,
                        args=[{'y':[df_comb_nonspam.Count_Reviews,df_comb_spam.Count_Reviews]}]
                        )
                  )
buttons.append(dict(method='restyle',
                        label='Average Overall Scores',
                        visible=True,
                        args=[{'y':[df_comb_nonspam.Overall_avg]}] 
                        )
                  )

# some adjustments to the updatemenus
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True

fig.update_layout(updatemenus=updatemenu, showlegend=True)
fig.show()

In [24]:
spam_reviewID_path = '/Users/pkpoon/Documents/School/Winter/msds697/Project/data/spam_reviewID_helpful.txt'

In [25]:
spam_reviewID = []
with open(spam_reviewID_path, 'r') as f:
    for line in f:
        spam_reviewID.append(line)
working = [line.split(",") for line in spam_reviewID]

In [26]:
spam_reviewID_lst = []
for item in working:
    spam_reviewID_lst.append([item[0][2], ''.join(re.findall(r"'([^']*)'", item[1])), item[2][1]])
spam_reviewID_lst[0]

['0', 'A3TKSUEV4E9PV6', '3']

In [27]:
working2 = []
for item in spam_reviewID_lst:
    if item[0] == '1':
        working2.append(['spam', item[1], item[2]])
    if item[0] == '0':
        working2.append(['not spam', item[1], item[2]])
df_spam_count_reviews = pd.DataFrame(working2,columns = ["Spam","ReviewerID","count_reviews"])
df_spam_count_reviews

,Spam,ReviewerID,count_reviews
0,not spam,A3TKSUEV4E9PV6,3
1,not spam,A563DWTSKZD49,1
2,not spam,A3J5L6UEFFMBG2,1
3,not spam,AH6PVAUDSPPG0,1
4,not spam,A368NHKLHMRNUV,1
...,...,...,...
1287375,not spam,A1X3RVTR23AEZX,1
1287376,not spam,A4KF54B7EPE2Q,1
1287377,not spam,AET38LB1SB7PT,1
1287378,not spam,AM4A6JLXB8YGE,1


In [34]:
fig = go.Figure()
fig.add_trace(
    go.bar(x=))

In [48]:
spam_full_path = '/Users/pkpoon/Documents/School/Winter/msds697/Project/data/spam_reviewID_helpful_full.txt'

In [49]:
spam_full = []
with open(spam_full_path, 'r') as f:
    for line in f:
        spam_full.append(line)
working = [line.split(",") for line in spam_full]
working[0]

['((1', " u'A1PHTLAKH5TDHR')", ' [0.0', ' 0.0])\n']

In [ ]:
for item in working:
    print(item[3])